## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [3]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [4]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [5]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [11]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [12]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962644100189),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.24438698589801788),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740755021572113),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222326040268)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [6]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [14]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-04-09 22:27:39--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-04-09 22:27:40--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2021-04-09 22:27:41--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Sa

In [15]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2021-04-09 22:27:45--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2021-04-09 22:27:46--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2021-04-09 22:27:47--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Savi

In [7]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [8]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [9]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression()
mapping.fit(X_train,Y_train)

LinearRegression()

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [10]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531402945518494),
 ('июнь', 0.8402308225631714),
 ('март', 0.8385774493217468),
 ('сентябрь', 0.8331867456436157),
 ('февраль', 0.8311492800712585),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244150280952454),
 ('июль', 0.8228995203971863),
 ('август', 0.8112362027168274),
 ('январь', 0.8022859692573547)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [11]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        tops  = ru_emb.most_similar([mapped_vectors[i]],topn=topn) 
        tops = [t[0] for t in tops]
        num_matches += ru in tops
    precision_val = num_matches / len(pairs)
    return precision_val

In [12]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [13]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [14]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [15]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [21]:
import numpy as np

In [22]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    # mapping = ...
    u, s, vh = np.linalg.svd(X_train.T @ Y_train)
    mapping = u @ vh
    return mapping

In [23]:
W = learn_transform(X_train, Y_train)

In [24]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.8056630492210388),
 ('сентябрь', 0.8055762648582458),
 ('март', 0.8032934665679932),
 ('октябрь', 0.7987101674079895),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.793963611125946),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.7715376019477844)]

In [25]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [70]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-04-09 23:52:11--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  2.47MB/s    in 0.7s    

2021-04-09 23:52:13 (2.47 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [71]:
!gzip -d ./uk.txt.gz

In [26]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [27]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [28]:
# Any necessary preprocessing if needed
# YOUR CODE HERE
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

preprocess = lambda text: " ".join(tokenizer.tokenize(text.lower()))

uk_corpus = [preprocess(txt) for txt in uk_corpus]

In [29]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    def to_ru(uk):
        if uk in uk_emb:
            return ru_emb.most_similar([np.matmul(uk_emb[uk], W)], topn = 1)[0][0]
        else:
            return '_'.join(["UNK", uk])
    
    translated = [to_ru(token_uk) for token_uk in sentence.split()]

    return " ".join(translated)

In [30]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [31]:
for sent in uk_corpus[::50]:
    print(f'uk: {sent}\nru: {translate(sent)}\n')

uk: я вже закінчу коледж , коли ви вернетеся з америки .
ru: мной уже закончу колледж , когда мы прибежишь со америки .

uk: подивися на той дим .
ru: посмотри по тот дым .

uk: у мене є мрія .
ru: во меня То мечта .

uk: він чекав на нього до десятої години .
ru: он ждал по него к десятой часа .

uk: я знаю , що ти багатий .
ru: мной знаю , что ты богатый .

uk: не твоя справа !
ru: не моя дело !

uk: коли ви народилися ?
ru: когда мы родились ?

uk: поверхня повітряної кулі — неевклідовий простір , тому для неї не виконуються правила евклідової геометрії .
ru: поверхность воздушной шары — UNK_неевклідовий пространство , потому для неё не выполняются правила симметрической геометрии .

uk: один , два , три , чотири , п'ять , шість , сім , вісім , дев'ять , десять .
ru: один , два , три , четыре , UNK_п'ять , восемь , семь , восемь , UNK_дев'ять , десять .

uk: кейт п ’ є багато молока кожен день .
ru: джастин аш со То много молока каждый день .

uk: я би з задоволенням написав сотні р

We see that there are a lot of unks and will use FastText based on n-grams (sub-words) [FastText](https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c)

In [32]:
import fasttext
import fasttext.util
import Levenshtein

#fasttext.util.download_model("uk", if_exists="ignore")
ft_uk = fasttext.load_model("cc.uk.300.bin")

In [33]:
def translate_ft(sentence, threshold = 30):
    """
    :args:
        sentence - sentence in Ukrainian (str)
        threshold - threshold for length work in more then threshold then UNK_*
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    def to_ru(uk):
        ru_token  = ru_emb.most_similar([np.matmul(ft_uk[uk], W)], topn = 1)[0][0]
        if(len(ru_token)>threshold):
            ru_token = '_'.join(["UNK", uk])
        return ru_token    
    
    translated = [to_ru(token_uk) for token_uk in sentence.split()]

    return " ".join(translated)

Let's compare model wod2vec with fasttext

In [43]:
ru_emb.most_similar([np.matmul(ft_uk['зловил'], W)], topn = 1)[0][0]

'поймал'

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb['зловил'], W)], topn = 1)[0][0]

Work2Vec can't find in vocabulary  'зловил' but fasttext can find similar work and return embedding and let's translate our corpus to ru

In [45]:
assert translate_ft(".") == "."
assert translate_ft("1 , 3") == "1 , 3"
assert translate_ft("кіт зловив мишу") == "кот поймал мышку"

In [46]:
for sent in uk_corpus[::50]:
    print(f'uk: {sent}\nru: {translate_ft(sent)}\n')

uk: я вже закінчу коледж , коли ви вернетеся з америки .
ru: мной уже закончу колледж , когда мы прибежишь со америки .

uk: подивися на той дим .
ru: посмотри по тот дым .

uk: у мене є мрія .
ru: во меня То мечта .

uk: він чекав на нього до десятої години .
ru: он ждал по него к десятой часа .

uk: я знаю , що ти багатий .
ru: мной знаю , что ты богатый .

uk: не твоя справа !
ru: не моя дело !

uk: коли ви народилися ?
ru: когда мы родились ?

uk: поверхня повітряної кулі — неевклідовий простір , тому для неї не виконуються правила евклідової геометрії .
ru: поверхность воздушной шары — математический пространство , потому для неё не выполняются правила симметрической геометрии .

uk: один , два , три , чотири , п'ять , шість , сім , вісім , дев'ять , десять .
ru: один , два , три , четыре , UNK_п'ять , восемь , семь , восемь , UNK_дев'ять , десять .

uk: кейт п ’ є багато молока кожен день .
ru: джастин аш со То много молока каждый день .

uk: я би з задоволенням написав сотні реч